## Homework

In [1]:
!wget --no-check-certificate 'https://docs.google.com/uc?export=download&id=1OKFSv2GpuUFDphO0r8LdM7bl6MAWwBfX' -O data.csv

--2022-05-21 21:37:04--  https://docs.google.com/uc?export=download&id=1OKFSv2GpuUFDphO0r8LdM7bl6MAWwBfX
Resolving docs.google.com (docs.google.com)... 173.194.214.138, 173.194.214.101, 173.194.214.102, ...
Connecting to docs.google.com (docs.google.com)|173.194.214.138|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://doc-04-ak-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/sq3vdrn8qgm2sj1cbejthpul56k2tsr1/1653168975000/03856158561714992485/*/1OKFSv2GpuUFDphO0r8LdM7bl6MAWwBfX?e=download [following]
--2022-05-21 21:37:06--  https://doc-04-ak-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/sq3vdrn8qgm2sj1cbejthpul56k2tsr1/1653168975000/03856158561714992485/*/1OKFSv2GpuUFDphO0r8LdM7bl6MAWwBfX?e=download
Resolving doc-04-ak-docs.googleusercontent.com (doc-04-ak-docs.googleusercontent.com)... 74.125.31.132, 2607:f8b0:400c:c02::84
Connecting to doc-04-ak-docs.googleusercontent.com (doc-04-ak-docs.g

В этой домашней работе вы будете предсказывать стоимость домов по их характеристикам.

Метрика качества: `RMSE`

Оценивание:
* Baseline - 2 балла
* Feature Engineering - 2 балла
* Model Selection - 3 балла
* Ensemble v.1 - 3 балла
* (*) Ensemble v.2 - дополнительно, 2 балла

### Описание датасета

Короткое описание данных:
```
price: sale price (this is the target variable)
id: transaction id
timestamp: date of transaction
full_sq: total area in square meters, including loggias, balconies and other non-residential areas
life_sq: living area in square meters, excluding loggias, balconies and other non-residential areas
floor: for apartments, floor of the building
max_floor: number of floors in the building
material: wall material
build_year: year built
num_room: number of living rooms
kitch_sq: kitchen area
state: apartment condition
product_type: owner-occupier purchase or investment
sub_area: name of the district

The dataset also includes a collection of features about each property's surrounding neighbourhood, and some features that are constant across each sub area (known as a Raion). Most of the feature names are self explanatory, with the following notes. See below for a complete list.

full_all: subarea population
male_f, female_f: subarea population by gender
young_*: population younger than working age
work_*: working-age population
ekder_*: retirement-age population
n_m_{all|male|female}: population between n and m years old
build_count_*: buildings in the subarea by construction type or year
x_count_500: the number of x within 500m of the property
x_part_500: the share of x within 500m of the property
_sqm_: square meters
cafe_count_d_price_p: number of cafes within d meters of the property that have an average bill under p RUB
trc_: shopping malls
prom_: industrial zones
green_: green zones
metro_: subway
_avto_: distances by car
mkad_: Moscow Circle Auto Road
ttk_: Third Transport Ring
sadovoe_: Garden Ring
bulvar_ring_: Boulevard Ring
kremlin_: City center
zd_vokzaly_: Train station
oil_chemistry_: Dirty industry
ts_: Power plant
```

### Setup

In [2]:
import pandas as pd
import numpy as np

In [3]:
df = pd.read_csv("data.csv", parse_dates=["timestamp"])
df

,id,timestamp,full_sq,life_sq,floor,max_floor,material,build_year,num_room,kitch_sq,...,cafe_count_5000_price_2500,cafe_count_5000_price_4000,cafe_count_5000_price_high,big_church_count_5000,church_count_5000,mosque_count_5000,leisure_count_5000,sport_count_5000,market_count_5000,price
0,0,2014-12-26,1,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,36,7,2,15,33,1,12,75,10,15318960
1,1,2012-10-04,64,64.0,16.0,NaN,NaN,NaN,NaN,NaN,...,2,2,0,0,13,1,0,6,1,6080000
2,2,2014-02-05,83,44.0,9.0,17.0,1.0,1985.0,3.0,10.0,...,13,6,1,8,18,0,1,52,0,17000000
3,3,2012-07-26,71,49.0,2.0,NaN,NaN,NaN,NaN,NaN,...,0,0,0,1,3,0,2,8,2,990000
4,4,2014-10-29,60,42.0,9.0,9.0,1.0,1970.0,3.0,6.0,...,3,1,0,5,8,0,1,34,5,7900000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19995,19995,2014-09-22,45,27.0,2.0,9.0,1.0,1978.0,2.0,5.0,...,9,0,0,6,29,0,1,46,10,7200000
19996,19996,2013-12-06,38,NaN,4.0,17.0,1.0,NaN,2.0,1.0,...,2,2,0,1,12,1,0,8,0,3971520
19997,19997,2014-06-26,35,14.0,9.0,22.0,1.0,2001.0,1.0,10.0,...,13,3,0,6,27,1,10,57,13,7200000
19998,19998,2014-01-21,51,30.0,8.0,17.0,1.0,2011.0,2.0,9.0,...,1,0,0,0,7,0,0,3,0,3300000


Разделите имеющиеся у вас данные на обучающую и тестовую выборки. В качестве обучающей выборки возьмите первые 80% данных, последние 20% - тестовая выборка.

In [4]:
drop_columns = [
    'id',           # May leak information
    'timestamp',    # May leak information
]
cat_columns = [
    'product_type',              #
    'material',                  # Material of the wall
    'state',                     # Satisfaction level
    'sub_area',                  # District name
    'culture_objects_top_25',    #
    'thermal_power_plant_raion', #
    'incineration_raion',        #
    'oil_chemistry_raion',       #
    'radiation_raion',           #
    'railroad_terminal_raion',   #
    'big_market_raion',          #
    'nuclear_reactor_raion',     #
    'detention_facility_raion',  #
    'ID_metro',                  #
    'ID_railroad_station_walk',  #
    'ID_railroad_station_avto',  #
    'water_1line',               #
    'ID_big_road1',              #
    'big_road1_1line',           #
    'ID_big_road2',              #
    'railroad_1line',            #
    'ID_railroad_terminal',      #
    'ID_bus_terminal',           #
    'ecology',                   #
]
num_columns = list(set(df.columns).difference(set(cat_columns + drop_columns)))

In [5]:
for column in num_columns: #Уберём nan в числовых признаках
  ser = df[column]
  ser = ser.dropna()
  df[column] = df[column].fillna(np.median(ser)) # Заменим на медианное значение
df

,id,timestamp,full_sq,life_sq,floor,max_floor,material,build_year,num_room,kitch_sq,...,cafe_count_5000_price_2500,cafe_count_5000_price_4000,cafe_count_5000_price_high,big_church_count_5000,church_count_5000,mosque_count_5000,leisure_count_5000,sport_count_5000,market_count_5000,price
0,0,2014-12-26,1,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,36,7,2,15,33,1,12,75,10,15318960
1,1,2012-10-04,64,64.0,16.0,12.0,NaN,1979.0,2.0,6.0,...,2,2,0,0,13,1,0,6,1,6080000
2,2,2014-02-05,83,44.0,9.0,17.0,1.0,1985.0,3.0,10.0,...,13,6,1,8,18,0,1,52,0,17000000
3,3,2012-07-26,71,49.0,2.0,12.0,NaN,1979.0,2.0,6.0,...,0,0,0,1,3,0,2,8,2,990000
4,4,2014-10-29,60,42.0,9.0,9.0,1.0,1970.0,3.0,6.0,...,3,1,0,5,8,0,1,34,5,7900000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19995,19995,2014-09-22,45,27.0,2.0,9.0,1.0,1978.0,2.0,5.0,...,9,0,0,6,29,0,1,46,10,7200000
19996,19996,2013-12-06,38,30.0,4.0,17.0,1.0,1979.0,2.0,1.0,...,2,2,0,1,12,1,0,8,0,3971520
19997,19997,2014-06-26,35,14.0,9.0,22.0,1.0,2001.0,1.0,10.0,...,13,3,0,6,27,1,10,57,13,7200000
19998,19998,2014-01-21,51,30.0,8.0,17.0,1.0,2011.0,2.0,9.0,...,1,0,0,0,7,0,0,3,0,3300000


In [6]:
from collections import Counter

for column in cat_columns: #Уберём nan в категориальных признаках
  ser = df[column]
  count = Counter(ser)
  df[column] = df[column].fillna(count.most_common()[0][0]) # Заменим на самое популярное значение значение
df

,id,timestamp,full_sq,life_sq,floor,max_floor,material,build_year,num_room,kitch_sq,...,cafe_count_5000_price_2500,cafe_count_5000_price_4000,cafe_count_5000_price_high,big_church_count_5000,church_count_5000,mosque_count_5000,leisure_count_5000,sport_count_5000,market_count_5000,price
0,0,2014-12-26,1,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,36,7,2,15,33,1,12,75,10,15318960
1,1,2012-10-04,64,64.0,16.0,12.0,1.0,1979.0,2.0,6.0,...,2,2,0,0,13,1,0,6,1,6080000
2,2,2014-02-05,83,44.0,9.0,17.0,1.0,1985.0,3.0,10.0,...,13,6,1,8,18,0,1,52,0,17000000
3,3,2012-07-26,71,49.0,2.0,12.0,1.0,1979.0,2.0,6.0,...,0,0,0,1,3,0,2,8,2,990000
4,4,2014-10-29,60,42.0,9.0,9.0,1.0,1970.0,3.0,6.0,...,3,1,0,5,8,0,1,34,5,7900000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19995,19995,2014-09-22,45,27.0,2.0,9.0,1.0,1978.0,2.0,5.0,...,9,0,0,6,29,0,1,46,10,7200000
19996,19996,2013-12-06,38,30.0,4.0,17.0,1.0,1979.0,2.0,1.0,...,2,2,0,1,12,1,0,8,0,3971520
19997,19997,2014-06-26,35,14.0,9.0,22.0,1.0,2001.0,1.0,10.0,...,13,3,0,6,27,1,10,57,13,7200000
19998,19998,2014-01-21,51,30.0,8.0,17.0,1.0,2011.0,2.0,9.0,...,1,0,0,0,7,0,0,3,0,3300000


In [12]:
from sklearn.model_selection import train_test_split


X = df.drop('price', axis=1)
y = df['price']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)
X

,id,timestamp,full_sq,life_sq,floor,max_floor,material,build_year,num_room,kitch_sq,...,mosque_count_5000,leisure_count_5000,sport_count_5000,market_count_5000,month_year_cnt,week_year_cnt,month,day_of_week,floor_ratio,area_ratio
0,0,2014-12-26,1,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1,12,75,10,1122,112,12,4,0.990099,0.990099
1,1,2012-10-04,64,64.0,16.0,12.0,1.0,1979.0,2.0,6.0,...,1,0,6,1,327,68,10,3,1.332223,0.093735
2,2,2014-02-05,83,44.0,9.0,17.0,1.0,1985.0,3.0,10.0,...,0,1,52,0,752,192,2,2,0.529101,0.120467
3,3,2012-07-26,71,49.0,2.0,12.0,1.0,1979.0,2.0,6.0,...,0,2,8,2,262,69,7,3,0.166528,0.084495
4,4,2014-10-29,60,42.0,9.0,9.0,1.0,1970.0,3.0,6.0,...,0,1,34,5,711,214,10,2,0.998890,0.099983
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19995,19995,2014-09-22,45,27.0,2.0,9.0,1.0,1978.0,2.0,5.0,...,0,1,46,10,734,181,9,0,0.221976,0.111086
19996,19996,2013-12-06,38,30.0,4.0,17.0,1.0,1979.0,2.0,1.0,...,1,0,8,0,579,153,12,4,0.235156,0.026309
19997,19997,2014-06-26,35,14.0,9.0,22.0,1.0,2001.0,1.0,10.0,...,1,10,57,13,861,277,6,3,0.408905,0.285633
19998,19998,2014-01-21,51,30.0,8.0,17.0,1.0,2011.0,2.0,9.0,...,0,0,3,0,573,164,1,1,0.470312,0.176436


Возможно в ваших моделях вам придется указывать, какие колонки являются категориальными (например, в бустингах). Для упрощения предлагается разделить колонки по следующему принципу:
```
drop_columns = [
    'id',           # May leak information
    'timestamp',    # May leak information
]
cat_columns = [
    'product_type',              #
    'material',                  # Material of the wall
    'state',                     # Satisfaction level
    'sub_area',                  # District name
    'culture_objects_top_25',    #
    'thermal_power_plant_raion', #
    'incineration_raion',        #
    'oil_chemistry_raion',       #
    'radiation_raion',           #
    'railroad_terminal_raion',   #
    'big_market_raion',          #
    'nuclear_reactor_raion',     #
    'detention_facility_raion',  #
    'ID_metro',                  #
    'ID_railroad_station_walk',  #
    'ID_railroad_station_avto',  #
    'water_1line',               #
    'ID_big_road1',              #
    'big_road1_1line',           #
    'ID_big_road2',              #
    'railroad_1line',            #
    'ID_railroad_terminal',      #
    'ID_bus_terminal',           #
    'ecology',                   #
]
num_columns = list(set(df.columns).difference(set(cat_columns + drop_columns)))
```

### Baseline (2 балла)

В качестве Baseline обучите `DecisionTreeRegressor` из `sklearn`.

In [24]:
from sklearn.tree import DecisionTreeRegressor
from copy import copy
import numpy as np

num_columns.remove('price')
num_columns_train = copy(num_columns)
num_columns.append('price')

X_train_now = X_train[num_columns_train]
X_test_now = X_test[num_columns_train]

X_train_now

,trc_sqm_3000,cafe_count_1000_price_1500,sport_count_2000,cafe_count_3000_price_2500,cafe_count_500_price_4000,build_count_1921-1945,mosque_count_5000,cafe_count_1000_na_price,7_14_female,cafe_count_5000_price_2500,...,trc_count_1000,big_church_count_2000,build_count_slag,sport_count_1500,work_all,green_part_1500,mosque_count_1000,0_13_all,church_count_5000,university_top_20_raion
0,772270,0,10,10,0,1.0,1,0,2535,36,...,1,1,0.0,4,49882,12.51,0,9319,33,0
1,0,0,1,1,0,2.0,1,0,305,2,...,0,0,0.0,1,6127,17.70,0,1207,13,0
2,405581,4,13,7,0,0.0,0,2,2545,13,...,5,1,0.0,9,48620,28.84,0,9168,18,0
3,34100,0,0,0,0,0.0,0,0,1270,0,...,0,0,0.0,0,22413,24.29,0,4828,3,0
4,441142,0,8,0,0,1.0,0,1,3852,3,...,0,2,0.0,4,98260,27.25,0,14694,8,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15995,240706,2,20,3,0,5.0,0,0,3241,9,...,1,0,4.0,9,91795,9.19,0,12162,19,0
15996,291635,2,8,2,0,0.0,1,1,2487,12,...,1,1,0.0,5,84387,8.78,0,10338,27,0
15997,176822,4,11,4,0,54.0,0,0,5164,7,...,0,1,2.0,7,140182,17.60,0,19389,26,0
15998,417410,0,5,0,0,1.0,0,0,3852,3,...,1,2,0.0,2,98260,14.15,0,14694,7,0


In [43]:
clf = DecisionTreeRegressor(max_depth=5, random_state=181)
clf.fit(X_train_now, y_train)

DecisionTreeRegressor(max_depth=5, random_state=181)

In [44]:
from sklearn.metrics import r2_score

r2_score(y_test, clf.predict(X_test_now))

0.37976368905220603

Проверьте качество на отложенной выборке.

In [45]:
for i in range(5, 15):
  clf = DecisionTreeRegressor(max_depth=i, random_state=181)
  clf.fit(X_train_now, y_train)
  print(f'max_depth: {i}, train: {r2_score(y_train, clf.predict(X_train_now))}, test: {r2_score(y_test, clf.predict(X_test_now))}')

max_depth: 5, train: 0.6266581718333295, test: 0.37976368905220603
max_depth: 6, train: 0.6712268788219781, test: 0.576938102631716
max_depth: 7, train: 0.71287520361159, test: 0.572148013592974
max_depth: 8, train: 0.7515360897030929, test: 0.40792713005654535
max_depth: 9, train: 0.7916794816066235, test: 0.39132066450969805
max_depth: 10, train: 0.8280670713865159, test: 0.35098492114593416
max_depth: 11, train: 0.8584921586593235, test: 0.3030052711761271
max_depth: 12, train: 0.8823301803000752, test: 0.5050290173554718
max_depth: 13, train: 0.9028648138115584, test: 0.4837420765840903
max_depth: 14, train: 0.9211622113966386, test: 0.26223719505923626


Лучшая на глубине 6-7, дальше идёт переобучение

### Feature Engineering (2 балла)

Часто улучшить модель можно с помощью аккуратного Feature Engineering.

Добавим в модель дополнительные признаки:
* "Как часто в этот год и этот месяц появлились объявления"
* "Как часто в этот год и эту неделю появлялись объявления"

In [7]:
month_year = (df.timestamp.dt.month + df.timestamp.dt.year * 100)
month_year_cnt_map = month_year.value_counts().to_dict()
df["month_year_cnt"] = month_year.map(month_year_cnt_map)

week_year = (df.timestamp.dt.weekofyear + df.timestamp.dt.year * 100)
week_year_cnt_map = week_year.value_counts().to_dict()
df["week_year_cnt"] = week_year.map(week_year_cnt_map)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated.  Please use Series.dt.isocalendar().week instead.
  """


Добавьте следюущие дополнительные признаки:
* Месяц (из колонки `timestamp`)
* День недели (из колонки `timestamp`)
* Отношение "этаж / максимальный этаж в здании" (колонки `floor` и `max_floor`)
* Отношение "площадь кухни / площадь квартиры" (колонки `kitchen_sq` и `full_sq`)

По желанию можно добавить и другие признаки.

In [8]:
df['month'] = df['timestamp'].dt.month
df['day_of_week'] = df['timestamp'].dt.day_of_week
df['floor_ratio'] = df['floor'] / (df['max_floor'] + 1e-2) # Чтобы не было inf
df['area_ratio'] = df['kitch_sq'] / (df['full_sq'] + 1e-2) # Чтобы не было inf
df

,id,timestamp,full_sq,life_sq,floor,max_floor,material,build_year,num_room,kitch_sq,...,leisure_count_5000,sport_count_5000,market_count_5000,price,month_year_cnt,week_year_cnt,month,day_of_week,floor_ratio,area_ratio
0,0,2014-12-26,1,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,12,75,10,15318960,1122,112,12,4,0.990099,0.990099
1,1,2012-10-04,64,64.0,16.0,12.0,1.0,1979.0,2.0,6.0,...,0,6,1,6080000,327,68,10,3,1.332223,0.093735
2,2,2014-02-05,83,44.0,9.0,17.0,1.0,1985.0,3.0,10.0,...,1,52,0,17000000,752,192,2,2,0.529101,0.120467
3,3,2012-07-26,71,49.0,2.0,12.0,1.0,1979.0,2.0,6.0,...,2,8,2,990000,262,69,7,3,0.166528,0.084495
4,4,2014-10-29,60,42.0,9.0,9.0,1.0,1970.0,3.0,6.0,...,1,34,5,7900000,711,214,10,2,0.998890,0.099983
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19995,19995,2014-09-22,45,27.0,2.0,9.0,1.0,1978.0,2.0,5.0,...,1,46,10,7200000,734,181,9,0,0.221976,0.111086
19996,19996,2013-12-06,38,30.0,4.0,17.0,1.0,1979.0,2.0,1.0,...,0,8,0,3971520,579,153,12,4,0.235156,0.026309
19997,19997,2014-06-26,35,14.0,9.0,22.0,1.0,2001.0,1.0,10.0,...,10,57,13,7200000,861,277,6,3,0.408905,0.285633
19998,19998,2014-01-21,51,30.0,8.0,17.0,1.0,2011.0,2.0,9.0,...,0,3,0,3300000,573,164,1,1,0.470312,0.176436


Разделите выборку на обучающую и тестовую еще раз (потому что дополнительные признаки созданы для исходной выборки).

In [48]:
X = df.drop('price', axis=1)
y = df['price']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)
X

,id,timestamp,full_sq,life_sq,floor,max_floor,material,build_year,num_room,kitch_sq,...,mosque_count_5000,leisure_count_5000,sport_count_5000,market_count_5000,month_year_cnt,week_year_cnt,month,day_of_week,floor_ratio,area_ratio
0,0,2014-12-26,1,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1,12,75,10,1122,112,12,4,0.990099,0.990099
1,1,2012-10-04,64,64.0,16.0,12.0,1.0,1979.0,2.0,6.0,...,1,0,6,1,327,68,10,3,1.332223,0.093735
2,2,2014-02-05,83,44.0,9.0,17.0,1.0,1985.0,3.0,10.0,...,0,1,52,0,752,192,2,2,0.529101,0.120467
3,3,2012-07-26,71,49.0,2.0,12.0,1.0,1979.0,2.0,6.0,...,0,2,8,2,262,69,7,3,0.166528,0.084495
4,4,2014-10-29,60,42.0,9.0,9.0,1.0,1970.0,3.0,6.0,...,0,1,34,5,711,214,10,2,0.998890,0.099983
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19995,19995,2014-09-22,45,27.0,2.0,9.0,1.0,1978.0,2.0,5.0,...,0,1,46,10,734,181,9,0,0.221976,0.111086
19996,19996,2013-12-06,38,30.0,4.0,17.0,1.0,1979.0,2.0,1.0,...,1,0,8,0,579,153,12,4,0.235156,0.026309
19997,19997,2014-06-26,35,14.0,9.0,22.0,1.0,2001.0,1.0,10.0,...,1,10,57,13,861,277,6,3,0.408905,0.285633
19998,19998,2014-01-21,51,30.0,8.0,17.0,1.0,2011.0,2.0,9.0,...,0,0,3,0,573,164,1,1,0.470312,0.176436


### Model Selection (3 балла)

Посмотрите, какого качества можно добиться если использовать разные модели:
* `DecisionTreeRegressor` из `sklearn`
* `RandomForestRegressor` из `sklearn`
* `CatBoostRegressor`

Также вы можете попробовать линейные модели, другие бустинги (`LigthGBM` и `XGBoost`).

Почти все библиотеки поддерживают удобный способ подбора гиперпараметров: посмотрите как это делать в [sklearn](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html) или в [catboost](https://catboost.ai/docs/concepts/python-reference_catboostregressor_grid_search.html).

Проверяйте качество каждой модели на тестовой выборке и выберите наилучшую.

In [49]:
# from sklearn.preprocessing import OneHotEncoder

# enc = OneHotEncoder()
# enc.fit_transform(df[cat_columns]).toarray()

In [50]:
# enc.get_feature_names_out()

Пока попробуем без cat_columns.

In [25]:
from sklearn.metrics import r2_score


num_columns_train.append('month')
num_columns_train.append('day_of_week')
num_columns_train.append('floor_ratio')
num_columns_train.append('area_ratio')

X_train_now = X_train[num_columns_train]
X_test_now = X_test[num_columns_train]

In [54]:
clf = DecisionTreeRegressor(max_depth=5, random_state=181)
clf.fit(X_train_now, y_train)
r2_score(y_test, clf.predict(X_test_now))

0.37976368905220603

In [55]:
for i in range(5, 15):
  clf = DecisionTreeRegressor(max_depth=i, random_state=181)
  clf.fit(X_train_now, y_train)
  print(f'max_depth: {i}, train: {r2_score(y_train, clf.predict(X_train_now))}, test: {r2_score(y_test, clf.predict(X_test_now))}')

max_depth: 5, train: 0.6266581718333295, test: 0.37976368905220603
max_depth: 6, train: 0.6712279922870419, test: 0.5767071578290889
max_depth: 7, train: 0.7138668655289202, test: 0.3802010398555178
max_depth: 8, train: 0.7541713611918892, test: 0.38154538599768495
max_depth: 9, train: 0.7909905647851211, test: 0.3608227874080461
max_depth: 10, train: 0.8233407608586714, test: 0.375194286944865
max_depth: 11, train: 0.8523902859719636, test: 0.5012802260955733
max_depth: 12, train: 0.8782965557948613, test: 0.3208695397399043
max_depth: 13, train: 0.9000519062843368, test: 0.4909309164610117
max_depth: 14, train: 0.9185457047270336, test: 0.463100768355025


Мб не надо было просто пребавлять 1e-2, но результат не стал хуже)
Попробуем сделать One-hot-encoding и посмотрим че получитсья


In [9]:
from sklearn.preprocessing import OneHotEncoder

enc = OneHotEncoder()
enc.fit_transform(df[cat_columns]).toarray()

array([[0., 1., 1., ..., 0., 1., 0.],
       [0., 1., 1., ..., 1., 0., 0.],
       [1., 0., 1., ..., 0., 0., 0.],
       ...,
       [1., 0., 1., ..., 0., 0., 0.],
       [1., 0., 1., ..., 1., 0., 0.],
       [1., 0., 1., ..., 0., 0., 1.]])

In [10]:
enc.get_feature_names_out()
df2 = pd.DataFrame(columns=enc.get_feature_names_out(), data=enc.fit_transform(df[cat_columns]).toarray())
df_new = pd.concat((df.drop(columns=cat_columns), df2), axis=1)
df_new

,id,timestamp,full_sq,life_sq,floor,max_floor,build_year,num_room,kitch_sq,area_m,...,ID_bus_terminal_10,ID_bus_terminal_11,ID_bus_terminal_12,ID_bus_terminal_13,ID_bus_terminal_14,ecology_excellent,ecology_good,ecology_no data,ecology_poor,ecology_satisfactory
0,0,2014-12-26,1,1.0,1.0,1.0,1.0,1.0,1.0,5.293465e+06,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,1,2012-10-04,64,64.0,16.0,12.0,1979.0,2.0,6.0,6.677245e+07,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2,2,2014-02-05,83,44.0,9.0,17.0,1985.0,3.0,10.0,1.216448e+07,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
3,3,2012-07-26,71,49.0,2.0,12.0,1979.0,2.0,6.0,4.708040e+06,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4,4,2014-10-29,60,42.0,9.0,9.0,1970.0,3.0,6.0,1.428699e+07,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19995,19995,2014-09-22,45,27.0,2.0,9.0,1978.0,2.0,5.0,2.481385e+07,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
19996,19996,2013-12-06,38,30.0,4.0,17.0,1979.0,2.0,1.0,6.677245e+07,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
19997,19997,2014-06-26,35,14.0,9.0,22.0,2001.0,1.0,10.0,1.004686e+07,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
19998,19998,2014-01-21,51,30.0,8.0,17.0,2011.0,2.0,9.0,4.036700e+07,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


In [78]:
X = df_new.drop(columns=['price'] + drop_columns, axis=1)
y = df_new['price']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)
X

,full_sq,life_sq,floor,max_floor,build_year,num_room,kitch_sq,area_m,raion_popul,green_zone_part,...,ID_bus_terminal_10,ID_bus_terminal_11,ID_bus_terminal_12,ID_bus_terminal_13,ID_bus_terminal_14,ecology_excellent,ecology_good,ecology_no data,ecology_poor,ecology_satisfactory
0,1,1.0,1.0,1.0,1.0,1.0,1.0,5.293465e+06,77878,0.023464,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,64,64.0,16.0,12.0,1979.0,2.0,6.0,6.677245e+07,9553,0.336177,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2,83,44.0,9.0,17.0,1985.0,3.0,10.0,1.216448e+07,78507,0.297166,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
3,71,49.0,2.0,12.0,1979.0,2.0,6.0,4.708040e+06,38075,0.271702,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4,60,42.0,9.0,9.0,1970.0,3.0,6.0,1.428699e+07,157010,0.389354,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19995,45,27.0,2.0,9.0,1978.0,2.0,5.0,2.481385e+07,174831,0.683844,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
19996,38,30.0,4.0,17.0,1979.0,2.0,1.0,6.677245e+07,9553,0.336177,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
19997,35,14.0,9.0,22.0,2001.0,1.0,10.0,1.004686e+07,76156,0.071056,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
19998,51,30.0,8.0,17.0,2011.0,2.0,9.0,4.036700e+07,21819,0.492840,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


In [79]:
for i in range(5, 15):
  clf = DecisionTreeRegressor(max_depth=i, random_state=181)
  clf.fit(X_train, y_train)
  print(f'max_depth: {i}, train: {r2_score(y_train, clf.predict(X_train))}, test: {r2_score(y_test, clf.predict(X_test))}')

max_depth: 5, train: 0.6266581718333295, test: 0.37976368905220603
max_depth: 6, train: 0.6713495919135919, test: 0.5738125588380139
max_depth: 7, train: 0.7125175298233457, test: 0.5853491685463685
max_depth: 8, train: 0.7530212406428619, test: 0.5467423610547834
max_depth: 9, train: 0.7931457160160912, test: 0.5090378392998841
max_depth: 10, train: 0.8279828473894606, test: 0.48411484933535087
max_depth: 11, train: 0.8584630180893844, test: 0.48023060856372213
max_depth: 12, train: 0.883562636115217, test: 0.26828182521310007
max_depth: 13, train: 0.9044711692538184, test: 0.23655545770790598
max_depth: 14, train: 0.9233868016977775, test: 0.2779150420249844


One-hot-encoding тоже не особо улучшил, ну мы попытались. Попробуем RandomForest

In [68]:
from sklearn.ensemble import RandomForestRegressor

X = df.drop('price', axis=1)
y = df['price']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)
X

,id,timestamp,full_sq,life_sq,floor,max_floor,material,build_year,num_room,kitch_sq,...,mosque_count_5000,leisure_count_5000,sport_count_5000,market_count_5000,month_year_cnt,week_year_cnt,month,day_of_week,floor_ratio,area_ratio
0,0,2014-12-26,1,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1,12,75,10,1122,112,12,4,0.990099,0.990099
1,1,2012-10-04,64,64.0,16.0,12.0,1.0,1979.0,2.0,6.0,...,1,0,6,1,327,68,10,3,1.332223,0.093735
2,2,2014-02-05,83,44.0,9.0,17.0,1.0,1985.0,3.0,10.0,...,0,1,52,0,752,192,2,2,0.529101,0.120467
3,3,2012-07-26,71,49.0,2.0,12.0,1.0,1979.0,2.0,6.0,...,0,2,8,2,262,69,7,3,0.166528,0.084495
4,4,2014-10-29,60,42.0,9.0,9.0,1.0,1970.0,3.0,6.0,...,0,1,34,5,711,214,10,2,0.998890,0.099983
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19995,19995,2014-09-22,45,27.0,2.0,9.0,1.0,1978.0,2.0,5.0,...,0,1,46,10,734,181,9,0,0.221976,0.111086
19996,19996,2013-12-06,38,30.0,4.0,17.0,1.0,1979.0,2.0,1.0,...,1,0,8,0,579,153,12,4,0.235156,0.026309
19997,19997,2014-06-26,35,14.0,9.0,22.0,1.0,2001.0,1.0,10.0,...,1,10,57,13,861,277,6,3,0.408905,0.285633
19998,19998,2014-01-21,51,30.0,8.0,17.0,1.0,2011.0,2.0,9.0,...,0,0,3,0,573,164,1,1,0.470312,0.176436


In [71]:
best = 0
for i in range(3, 8): #Всё равно дальше переобучение будет
  for j in range(47, 53):
    clf = RandomForestRegressor(n_estimators=j, max_depth=i, random_state=181)
    clf.fit(X_train_now, y_train)
    if (best < r2_score(y_test, clf.predict(X_test_now))):
      best = r2_score(y_test, clf.predict(X_test_now))
    print(f'max_depth: {i}, n_estimators: {j}, train: {r2_score(y_train, clf.predict(X_train_now))}, test: {r2_score(y_test, clf.predict(X_test_now))}')
print(f'best_test_r2: {best}')

max_depth: 3, n_estimators: 47, train: 0.5670723254710954, test: 0.5792856199932651
max_depth: 3, n_estimators: 48, train: 0.5670859760322788, test: 0.578922736992876
max_depth: 3, n_estimators: 49, train: 0.5672137354101452, test: 0.5787981152608257
max_depth: 3, n_estimators: 50, train: 0.5671703438597719, test: 0.578802260169679
max_depth: 3, n_estimators: 51, train: 0.5670930489317098, test: 0.5784262586403437
max_depth: 3, n_estimators: 52, train: 0.5669615455910204, test: 0.5785989471784533
max_depth: 4, n_estimators: 47, train: 0.622272302542876, test: 0.6129390662544174
max_depth: 4, n_estimators: 48, train: 0.6224588927639934, test: 0.6120003214275986
max_depth: 4, n_estimators: 49, train: 0.6223139794893549, test: 0.612359626295797
max_depth: 4, n_estimators: 50, train: 0.622041475591272, test: 0.6121886253812887
max_depth: 4, n_estimators: 51, train: 0.6223472306033005, test: 0.6118170987675107
max_depth: 4, n_estimators: 52, train: 0.6224506542495443, test: 0.61220337751545

In [75]:
X = df_new.drop(columns=['price'] + drop_columns, axis=1)
y = df_new['price']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)
X

,full_sq,life_sq,floor,max_floor,build_year,num_room,kitch_sq,area_m,raion_popul,green_zone_part,...,ID_bus_terminal_10,ID_bus_terminal_11,ID_bus_terminal_12,ID_bus_terminal_13,ID_bus_terminal_14,ecology_excellent,ecology_good,ecology_no data,ecology_poor,ecology_satisfactory
0,1,1.0,1.0,1.0,1.0,1.0,1.0,5.293465e+06,77878,0.023464,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,64,64.0,16.0,12.0,1979.0,2.0,6.0,6.677245e+07,9553,0.336177,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2,83,44.0,9.0,17.0,1985.0,3.0,10.0,1.216448e+07,78507,0.297166,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
3,71,49.0,2.0,12.0,1979.0,2.0,6.0,4.708040e+06,38075,0.271702,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4,60,42.0,9.0,9.0,1970.0,3.0,6.0,1.428699e+07,157010,0.389354,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19995,45,27.0,2.0,9.0,1978.0,2.0,5.0,2.481385e+07,174831,0.683844,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
19996,38,30.0,4.0,17.0,1979.0,2.0,1.0,6.677245e+07,9553,0.336177,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
19997,35,14.0,9.0,22.0,2001.0,1.0,10.0,1.004686e+07,76156,0.071056,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
19998,51,30.0,8.0,17.0,2011.0,2.0,9.0,4.036700e+07,21819,0.492840,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


In [77]:
best = 0
for i in range(3, 8): #Всё равно дальше переобучение будет
  for j in range(47, 53):
    clf = RandomForestRegressor(n_estimators=j, max_depth=i, random_state=181)
    clf.fit(X_train, y_train)
    if (best < r2_score(y_test, clf.predict(X_test))):
      best = r2_score(y_test, clf.predict(X_test))
    print(f'max_depth: {i}, n_estimators: {j}, train: {r2_score(y_train, clf.predict(X_train))}, test: {r2_score(y_test, clf.predict(X_test))}')
print(f'best_test_r2: {best}')

max_depth: 3, n_estimators: 47, train: 0.5670723254710954, test: 0.5792856199932651
max_depth: 3, n_estimators: 48, train: 0.5670859760322788, test: 0.578922736992876
max_depth: 3, n_estimators: 49, train: 0.5672137354101452, test: 0.5787981152608257
max_depth: 3, n_estimators: 50, train: 0.5671703438597719, test: 0.578802260169679
max_depth: 3, n_estimators: 51, train: 0.5670930489317098, test: 0.5784262586403437
max_depth: 3, n_estimators: 52, train: 0.5669615455910204, test: 0.5785989471784533
max_depth: 4, n_estimators: 47, train: 0.622878405619434, test: 0.6126807089686221
max_depth: 4, n_estimators: 48, train: 0.6230554809972584, test: 0.6117384784054765
max_depth: 4, n_estimators: 49, train: 0.6229011807616903, test: 0.6121110913390355
max_depth: 4, n_estimators: 50, train: 0.622621763785764, test: 0.6119445557543831
max_depth: 4, n_estimators: 51, train: 0.6229025239479856, test: 0.611554853803185
max_depth: 4, n_estimators: 52, train: 0.6230055010606289, test: 0.61197039438460

А здесь one-hot даже ухудшил

Ну и последняя деревянная модель: CatBoost

In [18]:
X = df.drop(columns=['price'] + drop_columns, axis=1)
y = df['price']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)
X

,full_sq,life_sq,floor,max_floor,material,build_year,num_room,kitch_sq,state,product_type,...,mosque_count_5000,leisure_count_5000,sport_count_5000,market_count_5000,month_year_cnt,week_year_cnt,month,day_of_week,floor_ratio,area_ratio
0,1,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,OwnerOccupier,...,1,12,75,10,1122,112,12,4,0.990099,0.990099
1,64,64.0,16.0,12.0,1.0,1979.0,2.0,6.0,2.0,OwnerOccupier,...,1,0,6,1,327,68,10,3,1.332223,0.093735
2,83,44.0,9.0,17.0,1.0,1985.0,3.0,10.0,3.0,Investment,...,0,1,52,0,752,192,2,2,0.529101,0.120467
3,71,49.0,2.0,12.0,1.0,1979.0,2.0,6.0,2.0,Investment,...,0,2,8,2,262,69,7,3,0.166528,0.084495
4,60,42.0,9.0,9.0,1.0,1970.0,3.0,6.0,2.0,Investment,...,0,1,34,5,711,214,10,2,0.998890,0.099983
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19995,45,27.0,2.0,9.0,1.0,1978.0,2.0,5.0,2.0,Investment,...,0,1,46,10,734,181,9,0,0.221976,0.111086
19996,38,30.0,4.0,17.0,1.0,1979.0,2.0,1.0,1.0,OwnerOccupier,...,1,0,8,0,579,153,12,4,0.235156,0.026309
19997,35,14.0,9.0,22.0,1.0,2001.0,1.0,10.0,2.0,Investment,...,1,10,57,13,861,277,6,3,0.408905,0.285633
19998,51,30.0,8.0,17.0,1.0,2011.0,2.0,9.0,2.0,Investment,...,0,0,3,0,573,164,1,1,0.470312,0.176436


In [22]:
for column in cat_columns:
  X_train[column] = X_train[column].astype(str)
  X_test[column] = X_test[column].astype(str)

In [23]:
#!pip install catboost
from catboost import CatBoostRegressor
from sklearn.metrics import r2_score

clf = CatBoostRegressor(cat_features=cat_columns)
clf.fit(X_train, y_train)
print(r2_score(y_test, clf.predict(X_test)))

Learning rate set to 0.06345
0:	learn: 4455314.2647768	total: 303ms	remaining: 5m 2s
1:	learn: 4320796.2679937	total: 418ms	remaining: 3m 28s
2:	learn: 4183476.9798880	total: 523ms	remaining: 2m 53s
3:	learn: 4070676.0567821	total: 626ms	remaining: 2m 35s
4:	learn: 3960727.4436243	total: 754ms	remaining: 2m 30s
5:	learn: 3867066.2434224	total: 861ms	remaining: 2m 22s
6:	learn: 3770543.8047981	total: 962ms	remaining: 2m 16s
7:	learn: 3683948.4026576	total: 1.07s	remaining: 2m 13s
8:	learn: 3603415.6645363	total: 1.19s	remaining: 2m 11s
9:	learn: 3529903.5673682	total: 1.3s	remaining: 2m 8s
10:	learn: 3460785.1933413	total: 1.4s	remaining: 2m 6s
11:	learn: 3398955.9417757	total: 1.5s	remaining: 2m 3s
12:	learn: 3341334.0558177	total: 1.63s	remaining: 2m 3s
13:	learn: 3291354.1749639	total: 1.73s	remaining: 2m 1s
14:	learn: 3243274.1646685	total: 1.82s	remaining: 1m 59s
15:	learn: 3200472.3961301	total: 1.93s	remaining: 1m 58s
16:	learn: 3159488.5059497	total: 2.05s	remaining: 1m 58s
17:	

Как видим это пока, что лучший результат. Так как catboost сам работает с категориальными признаками, то с one-hot-encoding пробовать не будем. А попробуем обычную Линейную регрессию

In [28]:
from sklearn.linear_model import LinearRegression

X_train_now = X_train[num_columns_train]
X_test_now = X_test[num_columns_train]

clf = LinearRegression()
clf.fit(X_train_now, y_train)
print(r2_score(y_test, clf.predict(X_test_now)))

0.43842759090992645


Ожидаемо, теперь попробуем с one-hot-encoding

In [29]:
X = df_new.drop(columns=['price'] + drop_columns, axis=1)
y = df_new['price']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)
X

,full_sq,life_sq,floor,max_floor,build_year,num_room,kitch_sq,area_m,raion_popul,green_zone_part,...,ID_bus_terminal_10,ID_bus_terminal_11,ID_bus_terminal_12,ID_bus_terminal_13,ID_bus_terminal_14,ecology_excellent,ecology_good,ecology_no data,ecology_poor,ecology_satisfactory
0,1,1.0,1.0,1.0,1.0,1.0,1.0,5.293465e+06,77878,0.023464,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,64,64.0,16.0,12.0,1979.0,2.0,6.0,6.677245e+07,9553,0.336177,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2,83,44.0,9.0,17.0,1985.0,3.0,10.0,1.216448e+07,78507,0.297166,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
3,71,49.0,2.0,12.0,1979.0,2.0,6.0,4.708040e+06,38075,0.271702,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4,60,42.0,9.0,9.0,1970.0,3.0,6.0,1.428699e+07,157010,0.389354,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19995,45,27.0,2.0,9.0,1978.0,2.0,5.0,2.481385e+07,174831,0.683844,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
19996,38,30.0,4.0,17.0,1979.0,2.0,1.0,6.677245e+07,9553,0.336177,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
19997,35,14.0,9.0,22.0,2001.0,1.0,10.0,1.004686e+07,76156,0.071056,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
19998,51,30.0,8.0,17.0,2011.0,2.0,9.0,4.036700e+07,21819,0.492840,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


In [30]:
clf = LinearRegression()
clf.fit(X_train, y_train)
print(r2_score(y_test, clf.predict(X_test)))

0.4328605264022849


Хммм, стало хуже. Ну да ладно

### Ensemble v.1 (3 балла)

Ансамбли иногда оказываются лучше чем одна большая модель.

В колонке `product_type` содержится информация о том, каким является объявление: `Investment` (продажа квартиры как инвестиции) или `OwnerOccupier` (продажа квартиры для жилья). Логично предположить, что если сделать по модели на каждый из этих типов, то качество будет выше.

Обучите свои лучшие модели на отдельно на `Investment` и `OwnerOccupier` (т.е. у вас будет `model_invest`, обученная на `(invest_train_X, invest_train_Y)` и `model_owner`, обученная на `(owner_train_X, owner_train_Y)`) и проверьте качество на отложенной выборке (т.е. на исходном `test_split`).

Так как CatBoost была лучше всех, то возьмём её

In [38]:
X = df.drop(columns=['price'] + drop_columns, axis=1)
y = df['price']
X_invest = X.loc[X['product_type'] == 'Investment']
X_owner = X.loc[X['product_type'] == 'OwnerOccupier']
y_invest = y.loc[df['product_type'] == 'Investment']
y_owner = y.loc[df['product_type'] == 'OwnerOccupier']
y_invest.shape, X_invest.shape

((12840,), (12840, 295))

In [39]:
for column in cat_columns:
  X_invest[column] = X_invest[column].astype(str)
  X_owner[column] = X_owner[column].astype(str)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [44]:
X_invest_train, X_invest_test, y_invest_train, y_invest_test = train_test_split(X_invest, y_invest, test_size=0.2, shuffle=False)
X_owner_train, X_owner_test, y_owner_train, y_owner_test = train_test_split(X_owner, y_owner, test_size=0.2, shuffle=False)
X_invest_train

,full_sq,life_sq,floor,max_floor,material,build_year,num_room,kitch_sq,state,product_type,...,mosque_count_5000,leisure_count_5000,sport_count_5000,market_count_5000,month_year_cnt,week_year_cnt,month,day_of_week,floor_ratio,area_ratio
2,83,44.0,9.0,17.0,1.0,1985.0,3.0,10.0,3.0,Investment,...,0,1,52,0,752,192,2,2,0.529101,0.120467
3,71,49.0,2.0,12.0,1.0,1979.0,2.0,6.0,2.0,Investment,...,0,2,8,2,262,69,7,3,0.166528,0.084495
4,60,42.0,9.0,9.0,1.0,1970.0,3.0,6.0,2.0,Investment,...,0,1,34,5,711,214,10,2,0.998890,0.099983
5,60,33.0,12.0,12.0,1.0,1979.0,2.0,6.0,2.0,Investment,...,1,1,65,7,550,50,4,1,0.999167,0.099983
6,59,43.0,4.0,9.0,1.0,1971.0,3.0,5.0,3.0,Investment,...,1,2,76,10,573,164,1,2,0.443951,0.084731
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16039,36,17.0,1.0,12.0,1.0,1979.0,2.0,6.0,2.0,Investment,...,1,32,137,16,262,52,7,0,0.083264,0.166620
16041,88,51.0,8.0,17.0,1.0,2004.0,3.0,12.0,3.0,Investment,...,0,33,135,13,579,32,12,0,0.470312,0.136348
16042,38,20.0,6.0,22.0,1.0,2004.0,1.0,10.0,3.0,Investment,...,0,0,29,4,861,187,6,2,0.272603,0.263089
16043,47,30.0,12.0,12.0,1.0,1977.0,2.0,6.0,2.0,Investment,...,0,2,44,11,1122,295,12,1,0.999167,0.127632


In [42]:
clf_invest = CatBoostRegressor(cat_features=cat_columns)
clf_invest.fit(X_invest_train, y_invest_train)
print(r2_score(y_invest_test, clf_invest.predict(X_invest_test)))

Learning rate set to 0.059159
0:	learn: 4755345.4533750	total: 155ms	remaining: 2m 34s
1:	learn: 4636998.0115021	total: 299ms	remaining: 2m 29s
2:	learn: 4517314.9606074	total: 454ms	remaining: 2m 30s
3:	learn: 4409413.3055072	total: 614ms	remaining: 2m 32s
4:	learn: 4313139.0044312	total: 798ms	remaining: 2m 38s
5:	learn: 4214124.3434952	total: 961ms	remaining: 2m 39s
6:	learn: 4123788.0368125	total: 1.14s	remaining: 2m 42s
7:	learn: 4043578.9120475	total: 1.28s	remaining: 2m 39s
8:	learn: 3966472.3892912	total: 1.46s	remaining: 2m 40s
9:	learn: 3897027.6248873	total: 1.62s	remaining: 2m 40s
10:	learn: 3833191.6335493	total: 1.77s	remaining: 2m 38s
11:	learn: 3778320.3119945	total: 1.89s	remaining: 2m 35s
12:	learn: 3720995.7115929	total: 2.07s	remaining: 2m 37s
13:	learn: 3670821.1247834	total: 2.22s	remaining: 2m 36s
14:	learn: 3628223.3271209	total: 2.39s	remaining: 2m 36s
15:	learn: 3574992.2137103	total: 2.51s	remaining: 2m 34s
16:	learn: 3531370.1311733	total: 2.7s	remaining: 2m

In [45]:
clf_owner = CatBoostRegressor(cat_features=cat_columns)
clf_owner.fit(X_owner_train, y_owner_train)
print(r2_score(y_owner_test, clf_owner.predict(X_owner_test)))

Learning rate set to 0.053944
0:	learn: 3838503.9796335	total: 73.1ms	remaining: 1m 12s
1:	learn: 3714173.0485749	total: 143ms	remaining: 1m 11s
2:	learn: 3592263.7882612	total: 224ms	remaining: 1m 14s
3:	learn: 3467350.8211702	total: 348ms	remaining: 1m 26s
4:	learn: 3346592.6006400	total: 479ms	remaining: 1m 35s
5:	learn: 3250802.2670019	total: 600ms	remaining: 1m 39s
6:	learn: 3141603.8655116	total: 706ms	remaining: 1m 40s
7:	learn: 3040812.1415034	total: 848ms	remaining: 1m 45s
8:	learn: 2951904.1742064	total: 965ms	remaining: 1m 46s
9:	learn: 2857515.1276751	total: 1.08s	remaining: 1m 47s
10:	learn: 2769021.5716467	total: 1.21s	remaining: 1m 49s
11:	learn: 2689063.6477493	total: 1.32s	remaining: 1m 48s
12:	learn: 2610454.2652233	total: 1.44s	remaining: 1m 49s
13:	learn: 2541584.7408996	total: 1.56s	remaining: 1m 50s
14:	learn: 2481657.6226738	total: 1.72s	remaining: 1m 52s
15:	learn: 2414720.0900809	total: 1.82s	remaining: 1m 52s
16:	learn: 2359014.8492855	total: 1.98s	remaining: 

In [67]:
X = df.drop(columns=['price'] + drop_columns, axis=1)
y = df['price']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)
X_test

,full_sq,life_sq,floor,max_floor,material,build_year,num_room,kitch_sq,state,product_type,...,mosque_count_5000,leisure_count_5000,sport_count_5000,market_count_5000,month_year_cnt,week_year_cnt,month,day_of_week,floor_ratio,area_ratio
16000,41,28.0,3.0,5.0,2.0,1965.0,2.0,6.0,2.0,Investment,...,0,2,33,6,859,112,4,0,0.598802,0.146306
16001,51,30.0,8.0,17.0,1.0,2002.0,2.0,9.0,3.0,Investment,...,0,0,40,4,547,141,8,4,0.470312,0.176436
16002,39,19.0,14.0,14.0,1.0,1973.0,1.0,10.0,2.0,Investment,...,2,3,68,12,859,210,4,3,0.999286,0.256345
16003,29,30.0,4.0,12.0,1.0,1979.0,2.0,6.0,2.0,OwnerOccupier,...,0,1,32,5,506,122,3,3,0.333056,0.206825
16004,65,40.0,4.0,12.0,1.0,1979.0,2.0,6.0,2.0,Investment,...,0,0,10,0,249,55,3,3,0.333056,0.092293
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19995,45,27.0,2.0,9.0,1.0,1978.0,2.0,5.0,2.0,Investment,...,0,1,46,10,734,181,9,0,0.221976,0.111086
19996,38,30.0,4.0,17.0,1.0,1979.0,2.0,1.0,1.0,OwnerOccupier,...,1,0,8,0,579,153,12,4,0.235156,0.026309
19997,35,14.0,9.0,22.0,1.0,2001.0,1.0,10.0,2.0,Investment,...,1,10,57,13,861,277,6,3,0.408905,0.285633
19998,51,30.0,8.0,17.0,1.0,2011.0,2.0,9.0,2.0,Investment,...,0,0,3,0,573,164,1,1,0.470312,0.176436


In [52]:
clf_invest.predict(X_invest_test).shape, 

(2568,)

In [69]:
for column in cat_columns:
  X_test[column] = X_test[column].astype(str)

In [72]:
y_pred = copy(y_test)
y_pred.loc[X_test['product_type'] == 'Investment'] = clf_invest.predict(X_test.loc[X_test['product_type'] == 'Investment'])
y_pred.loc[X_test['product_type'] != 'Investment'] = clf_owner.predict(X_test.loc[X_test['product_type'] != 'Investment'])
print(r2_score(y_test, y_pred))

0.7098468196795207


Удтвительно но стало чуть-чуть хуже. Однако owner отдельно предсказался хорошо

### (*) Ensemble v.2 (дополнительно, 2 балла)

Попробуйте сделать для `Investment` более сложную модель: обучите `CatBoostRegressor` и `HuberRegressor` из `sklearn`, а затем сложите их предсказания с весами `w_1` и `w_2` (выберите веса сами; сумма весов равняется 1).

Catboost уже обучен, осталось HuberRegressor

In [79]:
from sklearn.linear_model import HuberRegressor

hub_invest = HuberRegressor(max_iter=1000) # Так как one-hot-encoding так себе сегодня себя показывал, поэтому я просто выброшу cat_column

X_invest_train_now = X_invest_train[num_columns_train]
X_invest_test_now = X_invest_test[num_columns_train]

In [80]:
hub_invest.fit(X_invest_train_now, y_invest_train)
print(r2_score(y_invest_test, hub_invest.predict(X_invest_test_now)))

-0.09370656936546973


Мдааа, хуже чем у константы(так как у r2 константа всегда 0). Однако мы не подобрали веса, но понятно, что у Catboost он долже быть больше. Ну это логично так как мы просто заигнорили категориальные признаки

In [89]:
best = 0
coef = 0
for w1000 in range(0, 101, 1):
  w = w1000 / 100
  y_pred = w * clf_invest.predict(X_invest_test) + (1 - w) * hub_invest.predict(X_invest_test_now)
  if (best < r2_score(y_invest_test, y_pred)):
    best = r2_score(y_invest_test, y_pred)
    coef = w
print(f'best: {best}, cat_coef: {coef}, hug_coef: {1 - coef}')

best: 0.649743015959884, cat_coef: 0.98, hug_coef: 0.020000000000000018


Как я и говорил у CatBoost почти 1. Однако не 1, значит мы чуть чуть улучшили

In [90]:
y_pred = copy(y_test)
X_test_now = X_test[num_columns_train]
y_pred.loc[X_test['product_type'] == 'Investment'] = (coef * clf_invest.predict(X_test.loc[X_test['product_type'] == 'Investment']) +
                                                      (1 - coef) * hub_invest.predict(X_test_now.loc[X_test['product_type'] == 'Investment']))
y_pred.loc[X_test['product_type'] != 'Investment'] = clf_owner.predict(X_test.loc[X_test['product_type'] != 'Investment'])
print(r2_score(y_test, y_pred))

0.7100493262073846


Действительно стало лучше. Однако лучшее всего ровно чистый catBoost на всём датасете